<div class="alert alert-block alert-info">
This script imports WIOD for exploration. No outputs...<br>  
    <hr> 
    Note: <br>
    <i><b>Not a part of the pipeline</b></i>
</div>

# Imports

In [1]:
%config IPCompleter.use_jedi = False # disable jedi autocompleter (https://stackoverflow.com/a/65734178/14485040)

import project_path  # makes possible the access to `src` directory using relative path
from src.utils import explore_dir
from src.utils import read_excel_to_pandas as r_excel

%run init_nb.ipynb

# INPUTS: Identify file(s) and read data to df

In [2]:
# Explore the directory to find the file(s)

# --------------- WorldInputOutputTable (WIOT) file ----------------
inputs_dir_1, files_list_1 = explore_dir(
    path_to_dir=r"..\data\external\WIOD",
    file_extension="xlsb",
    print_files_list=True,
)

print("\nnº of found files in", inputs_dir_1, ": ", len(files_list_1), "\n")

['WIOT2014_Nov16_ROW.xlsb']

nº of found files in C:\Users\ViteksPC\Documents\00-ETH_projects\17-AESA_ecoinvent_chemicals\data\external\WIOD :  1 



In [3]:
%%time
# Process raw data

# IO data for a specific year
df_IO_raw = r_excel(
    inputs_dir_1,
    "WIOT2014_Nov16_ROW.xlsb",
    sheets="2014",
    engine="pyxlsb",
    index_col=[0, 1, 2, 3],
    header=[2, 3, 4, 5],
#     skiprows = [0, 1]
)
print("df of WIOT (raw) ".ljust(40, "."), f" {df_IO_raw.shape}".rjust(13, "."))
df_IO_raw.head(5)

df of WIOT (raw) .......................  (2472, 2685)
Wall time: 1min 21s


,,,(industry-by-industry),A01,A02,A03,B,C10-C12,C13-C15,C16,C17,C18,C19,...,CONS_np,CONS_g,GFCF,INVEN,CONS_h,CONS_np,CONS_g,GFCF,INVEN,GO
,,,(millions of US$),"Crop and animal production, hunting and related service activities",Forestry and logging,Fishing and aquaculture,Mining and quarrying,"Manufacture of food products, beverages and tobacco products","Manufacture of textiles, wearing apparel and leather products","Manufacture of wood and of products of wood and cork, except furniture; manufacture of articles of straw and plaiting materials",Manufacture of paper and paper products,Printing and reproduction of recorded media,Manufacture of coke and refined petroleum products,...,Final consumption expenditure by non-profit organisations serving households (NPISH),Final consumption expenditure by government,Gross fixed capital formation,Changes in inventories and valuables,Final consumption expenditure by households,Final consumption expenditure by non-profit organisations serving households (NPISH),Final consumption expenditure by government,Gross fixed capital formation,Changes in inventories and valuables,Total output
,,,,AUS,AUS,AUS,AUS,AUS,AUS,AUS,AUS,AUS,AUS,...,USA,USA,USA,USA,ROW,ROW,ROW,ROW,ROW,TOT
,,,,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,...,c58,c59,c60,c61,c57,c58,c59,c60,c61,c62
A01,"Crop and animal production, hunting and related service activities",AUS,r1,12924.179691,112.454221,228.480462,538.890807,25139.334287,737.492591,4.475430,3.130541,1.889580,0.226825,...,0,3.565392e-05,0.324914,0.027391,723.189984,0.000374,14.156350,20.520708,0.010083,70292.034492
A02,Forestry and logging,AUS,r2,83.029637,201.499948,0.191313,17.190679,2.646331,0.093768,1430.804904,0.369454,4.934620,0.251875,...,0,1.647680e-06,0.015015,0.000439,0.423422,0.000003,0.000071,0.000977,-0.000109,2585.379685
A03,Fishing and aquaculture,AUS,r3,19.147731,0.001079,19.106701,4.228108,316.075149,22.866683,0.252294,0.268537,0.298600,0.036279,...,0,5.445825e-07,0.004963,0.008311,404.041139,0.000386,0.004619,0.357978,0.000592,3175.044396
B,Mining and quarrying,AUS,r4,115.929847,0.695040,5.924477,4492.037652,331.609356,5.984141,29.740924,49.763961,21.382742,10759.596823,...,0,1.767029e-04,1.610294,0.047027,14.871627,0.006665,0.594705,19.726833,-0.282072,171985.122327
C10-C12,"Manufacture of food products, beverages and tobacco products",AUS,r5,1590.840590,0.880063,42.540760,203.011938,10851.759013,262.228559,11.467425,18.140557,10.902817,2.897519,...,0,1.528107e-04,1.393463,0.041279,4810.234988,0.001371,47.016724,1.345248,0.027914,83504.036584


# Operations

 - aggregate WIOD by sector for one GLO country (aggregate rows and columns)
 - put rows/columns in order (e.g. r9 comes before r10)
 - separate WIOD into T, Y, X, VA matrices
 - perform operations

In [4]:
# clean column/index names
df_IO_raw.columns.names = len(df_IO_raw.columns.names)*[None]
df_IO_raw.index.names = len(df_IO_raw.index.names)*[None]
df_io = df_IO_raw.copy()

In [5]:
io_cols = pd.DataFrame(df_io.columns.to_list(), columns=["col_code", "sector", "country", "col_num"])
io_cols

,col_code,sector,country,col_num
0,A01,"Crop and animal production, hunting and relate...",AUS,c1
1,A02,Forestry and logging,AUS,c2
2,A03,Fishing and aquaculture,AUS,c3
3,B,Mining and quarrying,AUS,c4
4,C10-C12,"Manufacture of food products, beverages and to...",AUS,c5
...,...,...,...,...
2680,CONS_np,Final consumption expenditure by non-profit or...,ROW,c58
2681,CONS_g,Final consumption expenditure by government,ROW,c59
2682,GFCF,Gross fixed capital formation,ROW,c60
2683,INVEN,Changes in inventories and valuables,ROW,c61


In [6]:
io_rows = pd.DataFrame(df_io.index.to_list(), columns=["row_code", "sector", "country", "row_num"])
io_rows

,row_code,sector,country,row_num
0,A01,"Crop and animal production, hunting and relate...",AUS,r1
1,A02,Forestry and logging,AUS,r2
2,A03,Fishing and aquaculture,AUS,r3
3,B,Mining and quarrying,AUS,r4
4,C10-C12,"Manufacture of food products, beverages and to...",AUS,r5
...,...,...,...,...
2467,PURR,Direct purchases abroad by residents,TOT,r68
2468,PURNR,Purchases on the domestic territory by non-res...,TOT,r69
2469,VA,Value added at basic prices,TOT,r70
2470,IntTTM,International Transport Margins,TOT,r71


In [7]:
map_col_num_to_code = dict()
for i in pd.DataFrame(io_cols.groupby(["col_code", "sector", "col_num"]))[0]:
    map_col_num_to_code[i[2]] = [i[0], i[1]]
# map_col_num_to_code

In [8]:
df_col_num_to_code = pd.DataFrame(
    map_col_num_to_code, index=[["col_code", "col_name"]]
).T
df_col_num_to_code = df_col_num_to_code.loc[
    sorted(df_col_num_to_code.index.to_list(), key=lambda x: int(x.partition("c")[2]))
]
df_col_num_to_code.tail(10)

,col_code,col_name
c53,Q,Human health and social work activities
c54,R_S,Other service activities
c55,T,Activities of households as employers; undiffe...
c56,U,Activities of extraterritorial organizations a...
c57,CONS_h,Final consumption expenditure by households
c58,CONS_np,Final consumption expenditure by non-profit or...
c59,CONS_g,Final consumption expenditure by government
c60,GFCF,Gross fixed capital formation
c61,INVEN,Changes in inventories and valuables
c62,GO,Total output


In [9]:
map_row_num_to_code = dict()
for i in pd.DataFrame(io_rows.groupby(["row_code", "sector", "row_num"]))[0]:
    map_row_num_to_code[i[2]] = [i[0], i[1]]
# map_row_num_to_code

In [10]:
df_row_num_to_code = pd.DataFrame(
    map_row_num_to_code, index=[["row_code", "row_name"]]
).T
df_row_num_to_code = df_row_num_to_code.loc[
    sorted(df_row_num_to_code.index.to_list(), key=lambda x: int(x.partition("r")[2]))
]
df_row_num_to_code.tail(10)

,row_code,row_name
r55,T,Activities of households as employers; undiffe...
r56,U,Activities of extraterritorial organizations a...
r65,II_fob,Total intermediate consumption
r66,TXSP,taxes less subsidies on products
r67,EXP_adj,Cif/ fob adjustments on exports
r68,PURR,Direct purchases abroad by residents
r69,PURNR,Purchases on the domestic territory by non-res...
r70,VA,Value added at basic prices
r71,IntTTM,International Transport Margins
r73,GO,Output at basic prices


In [11]:
# io_rows[~io_rows.row_code.isin(io_cols.col_code)]

In [12]:
# io_cols[~io_cols.col_code.isin(io_rows.row_code)]
# pd.DataFrame(io_cols[~io_cols.col_code.isin(io_rows.row_code)].groupby(by="col_code"))

In [13]:
len(map_row_num_to_code)

64

In [14]:
len(map_col_num_to_code)

62

In [15]:
# Aggregate columns by col_num and rows by row_num
# This will create a GLO IO table industry-by-industry
df_io_GLO_raw = df_io.groupby(axis=1, level=3).sum().groupby(axis=0, level=3).sum()

# sort columns/rows by col_num/row_num in ascending order
df_io_GLO_raw = df_io_GLO_raw.loc[
    sorted(df_io_GLO_raw.index.to_list(), key=lambda x: int(x.partition("r")[2])),
    sorted(df_io_GLO_raw.columns.to_list(), key=lambda x: int(x.partition("c")[2])),
]
# df_io_GLO_raw

# aggregated IO table with names of sectors
df_io_GLO_name = df_io_GLO_raw.rename(index = {k:v[1] for k,v in map_row_num_to_code.items()},
                     columns = {k:v[1] for k,v in map_col_num_to_code.items()})

# aggregated IO table with codes of sectors
df_io_GLO_code = df_io_GLO_raw.rename(index = {k:v[0] for k,v in map_row_num_to_code.items()},
                     columns = {k:v[0] for k,v in map_col_num_to_code.items()})

In [16]:
# matrix T of domestic transactions
df_T = df_io_GLO_raw.loc[
    df_io_GLO_raw.index[
        ~df_io_GLO_raw.index.isin(["r65", "r66", "r67", "r68", "r69", "r70", "r71", "r73"])
    ],    
    df_io_GLO_raw.columns[
        ~df_io_GLO_raw.columns.isin(["c57", "c58", "c59", "c60", "c61", "c62"])
    ],
]

In [17]:
# matrix Y of Final Demands
df_FDs = df_io_GLO_raw.loc[
    df_io_GLO_raw.index[
        ~df_io_GLO_raw.index.isin(["r65", "r66", "r67", "r68", "r69", "r70", "r71", "r73"])
    ],    
    df_io_GLO_raw.columns[
        df_io_GLO_raw.columns.isin(["c57", "c58", "c59", "c60", "c61"]) # , "c62"
    ],
]
df_Y_temp = df_FDs.sum(axis=1) # aggregate all Final demands in one vector-column
df_Y = pd.DataFrame(df_Y_temp, columns=["Y"])

In [18]:
# matrix X of gross output
df_X = df_io_GLO_raw.loc[
    df_io_GLO_raw.index[
        ~df_io_GLO_raw.index.isin(["r65", "r66", "r67", "r68", "r69", "r70", "r71", "r73"])
    ],    
    df_io_GLO_raw.columns[
        df_io_GLO_raw.columns.isin(["c62"])
    ],
]
# df_X

In [19]:
# vector VA of value added
df_VA = df_io_GLO_raw.loc[
    df_io_GLO_raw.index[
        df_io_GLO_raw.index.isin(["r70"])
    ],    
    df_io_GLO_raw.columns[
        ~df_io_GLO_raw.columns.isin(["c57", "c58", "c59", "c60", "c61", "c62"])
    ],
] 
# df_VA

In [20]:
# total GLO value added at basic prices
totGVA = df_VA.sum(axis=1)

In [21]:
# share allocated to a sector : VAsector/totVA
df_share_sectors = df_VA.div(totGVA[0])

In [22]:
# map_col_num_to_code["c11"]

In [23]:
df_col_num_to_code.loc["c11"]

col_code                                                C20
col_name    Manufacture of chemicals and chemical products 
Name: c11, dtype: object

In [24]:
list_sectors = ["c10", "c11", "c12", "c13"]  # sectors to include?
df_col_num_to_code.loc[list_sectors]

,col_code,col_name
c10,C19,Manufacture of coke and refined petroleum prod...
c11,C20,Manufacture of chemicals and chemical products
c12,C21,Manufacture of basic pharmaceutical products a...
c13,C22,Manufacture of rubber and plastic products


In [25]:
list_sectors_rows = ["r10", "r11", "r12", "r13"]  # sectors to include?
df_row_num_to_code.loc[list_sectors_rows]

,row_code,row_name
r10,C19,Manufacture of coke and refined petroleum prod...
r11,C20,Manufacture of chemicals and chemical products
r12,C21,Manufacture of basic pharmaceutical products a...
r13,C22,Manufacture of rubber and plastic products


## Gonzalo's calculations

In [26]:
# df_T[list_sectors].sum(axis=1)

In [27]:
# fraction of what each sector gives to sector c11 relative to the total output
frac_spr_to_s = pd.DataFrame(df_T["c11"].to_numpy()/df_X.iloc[:,0].to_numpy().T, index=df_T.index, columns=["s_out_per_X"])
# frac_spr_to_s

delta_share_spr_to_s = pd.DataFrame(
    df_share_sectors.to_numpy() * frac_spr_to_s.to_numpy().T,
    index=["delta_share"],
    columns=df_share_sectors.columns,
)
delta_share_spr_to_s["c11"] = 0 # sector doesn't give a share to itself
delta_share_spr_to_s

,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,...,c47,c48,c49,c50,c51,c52,c53,c54,c55,c56
delta_share,0.000301,0.000092,0.000004,0.001727,0.000333,0.000068,0.000024,0.000083,0.00003,0.000721,...,0.000064,0.000068,0.000139,0.000246,0.000102,0.000026,0.000012,0.000097,0.000007,0.0


In [28]:
print("direct share =",df_share_sectors["c11"][0]) 
print("deltashare = ",delta_share_spr_to_s.sum(axis=1)[0])
share_c11 =  df_share_sectors["c11"] + delta_share_spr_to_s.sum(axis=1)[0]
share_c11

direct share = 0.013149552186725817
deltashare =  0.010539745448773237


r70    0.023689
Name: c11, dtype: float64

#### aggregated sectors c10-13

In [29]:
# df_io_GLO_raw.loc[list_sectors_rows].sum(axis=0)

In [30]:
df_io_GLO_raw_aggsectors = df_io_GLO_raw.copy()
df_io_GLO_raw_aggsectors["c10-13"] = df_io_GLO_raw_aggsectors[list_sectors].sum(axis=1)
df_io_GLO_raw_aggsectors.loc["r10-13"] = df_io_GLO_raw_aggsectors.loc[
    list_sectors_rows
].sum(axis=0)
df_io_GLO_raw_aggsectors.drop(list_sectors, axis=1, inplace=True)
df_io_GLO_raw_aggsectors.drop(list_sectors_rows, axis=0, inplace=True)
df_io_GLO_raw_aggsectors.rename(columns={"c10-13": "c10"}, inplace=True)
df_io_GLO_raw_aggsectors.rename(index={"r10-13": "r10"}, inplace=True)

df_io_GLO_raw_aggsectors = df_io_GLO_raw_aggsectors.loc[
    sorted(df_io_GLO_raw_aggsectors.index.to_list(), key=lambda x: int(x.partition("r")[2])),
    sorted(df_io_GLO_raw_aggsectors.columns.to_list(), key=lambda x: int(x.partition("c")[2])),]
# df_io_GLO_raw_aggsectors

In [31]:
df_T_aggsectors = df_io_GLO_raw_aggsectors.loc[
    df_io_GLO_raw_aggsectors.index[
        ~df_io_GLO_raw_aggsectors.index.isin(["r65", "r66", "r67", "r68", "r69", "r70", "r71", "r73"])
    ],    
    df_io_GLO_raw_aggsectors.columns[
        ~df_io_GLO_raw_aggsectors.columns.isin(["c57", "c58", "c59", "c60", "c61", "c62"])
    ],
]
# df_T_aggsectors

In [32]:
# check...
df_X.loc[list_sectors_rows].sum(axis=0)

c62    1.112437e+07
dtype: float64

In [33]:
df_X_aggsectors = df_io_GLO_raw_aggsectors.loc[
    df_io_GLO_raw_aggsectors.index[
        ~df_io_GLO_raw_aggsectors.index.isin(["r65", "r66", "r67", "r68", "r69", "r70", "r71", "r73"])
    ],    
    df_io_GLO_raw_aggsectors.columns[
        df_io_GLO_raw_aggsectors.columns.isin(["c62"])
    ],
]
# df_X_aggsectors #.loc["r10"]

In [34]:
# check...
df_VA[list_sectors].sum(axis=1)

r70    2.434652e+06
dtype: float64

In [35]:
# vector VA of value added
df_VA_aggsectors = df_io_GLO_raw_aggsectors.loc[
    df_io_GLO_raw_aggsectors.index[
        df_io_GLO_raw_aggsectors.index.isin(["r70"])
    ],    
    df_io_GLO_raw_aggsectors.columns[
        ~df_io_GLO_raw_aggsectors.columns.isin(["c57", "c58", "c59", "c60", "c61", "c62"])
    ],
] 
# df_VA_aggsectors # ["c10"]

In [36]:
df_share_sectors[list_sectors].sum(axis=1)

r70    0.032987
dtype: float64

In [37]:
totGVA_aggsectors = df_VA_aggsectors.sum(axis=1)
df_share_sectors_aggsectors = df_VA_aggsectors.div(totGVA_aggsectors[0])
# df_share_sectors_aggsectors # ["c10"]

In [38]:
# fraction of what each sector gives to sector c11 relative to the total output
frac_spr_to_s_aggsectors = pd.DataFrame(
    df_T_aggsectors["c10"].to_numpy() / df_X_aggsectors.iloc[:, 0].to_numpy().T,
    index=df_T_aggsectors.index,
    columns=["s_out_per_X"],
)
# frac_spr_to_s

delta_share_spr_to_s_aggsectors = pd.DataFrame(
    df_share_sectors_aggsectors.to_numpy() * frac_spr_to_s_aggsectors.to_numpy().T,
    index=["delta_share"],
    columns=df_share_sectors_aggsectors.columns,
)
delta_share_spr_to_s_aggsectors["c10"] = 0  # sector doesn't give a share to itself
delta_share_spr_to_s_aggsectors

,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,...,c47,c48,c49,c50,c51,c52,c53,c54,c55,c56
delta_share,0.00107,0.000272,0.000014,0.017738,0.000505,0.000244,0.000057,0.000181,0.000064,0,...,0.000158,0.000151,0.000234,0.000721,0.000301,0.000068,0.000039,0.000238,0.000024,0.0


In [39]:
print("direct share =", df_share_sectors_aggsectors["c10"][0])
print("deltashare = ", delta_share_spr_to_s_aggsectors.sum(axis=1)[0])
share_aggsectors =  df_share_sectors_aggsectors["c10"] + delta_share_spr_to_s_aggsectors.sum(axis=1)[0]
share_aggsectors

direct share = 0.032986778612954
deltashare =  0.035932132523727706


r70    0.068919
Name: c10, dtype: float64

## Calculations using L inverse

In [40]:
# check that T + Y = X
a = df_T.sum(axis=1) + df_Y.sum(axis=1)
a.to_numpy()/df_X.to_numpy().T[0]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.])

In [41]:
# diagonalize X
diagX_raw = np.diag(df_X.to_numpy().T[0])
diagX = np.where(diagX_raw==0, 1e-100, diagX_raw) # small non-zero values, to be able to invert this matrix

In [42]:
#calculate technical coefficients A
df_A = pd.DataFrame(df_T.to_numpy() @ np.linalg.inv(diagX), index=df_T.index, columns=df_T.columns)

In [43]:
# calculate Leontief's inverse matrix
IminusA = np.eye(df_A.shape[0]) - df_A
df_L = pd.DataFrame(np.linalg.inv(IminusA.to_numpy()), index=df_A.index, columns=df_A.columns)
# df_L

In [44]:
# alternative method!

# share intensities : df_share_sectors / X
share_intensity = pd.DataFrame(
    df_share_sectors.iloc[0, :].to_numpy() / df_X.iloc[:, 0].to_numpy(),
    index=df_share_sectors.columns, columns=["share_intensity"]
).T
share_intensity

,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,...,c47,c48,c49,c50,c51,c52,c53,c54,c55,c56
share_intensity,7.795328e-09,7.584964e-09,8.460434e-09,7.597454e-09,3.347104e-09,3.243496e-09,3.659740e-09,3.496974e-09,5.156045e-09,1.940033e-09,...,7.238760e-09,5.983232e-09,7.505347e-09,8.314519e-09,8.826940e-09,9.602517e-09,7.838411e-09,7.579834e-09,1.049042e-08,2.135453e-09


In [45]:
# check, production based intensities @ gross output should be equal to the total share (which is 1)
share_intensity.to_numpy() @ df_X.to_numpy()
# this has to be equal to the shares consumption based (below)

array([[1.]])

In [46]:
# calculate the share (consumption based) : share_intensity * L * df_Y
# share_consumption_based = share_intensity.to_numpy() @ df_L.to_numpy() @ df_Y.to_numpy()
share_consumption_based = pd.DataFrame(
    share_intensity.to_numpy() @ df_L.to_numpy() * df_Y.to_numpy().T,
    index=["share_cons_base"],
    columns=share_intensity.columns,
)
share_consumption_based #.sum(axis=1)

,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,...,c47,c48,c49,c50,c51,c52,c53,c54,c55,c56
share_cons_base,0.022961,0.000813,0.002716,0.00514,0.04941,0.016103,0.001003,0.001449,0.000713,0.011717,...,0.005464,0.001059,0.003061,0.01036,0.106316,0.045123,0.081208,0.03034,0.001983,0.000001


In [47]:
share_consumption_based["c11"]

share_cons_base    0.007607
Name: c11, dtype: float64

In [48]:
print("share of sectors = ", share_consumption_based[list_sectors].sum(axis=1)[0])
share_consumption_based[list_sectors].sum(axis=1)

share of sectors =  0.02736665572929991


share_cons_base    0.027367
dtype: float64

In [49]:
# GVA_chemsector = 9.71e11
# GVA_world = 7.38e13
# deltaSOS_purchases = 0.0234371088889808
# GVA_chemsector / GVA_world + deltaSOS_purchases

In [50]:
df_share_sectors[list_sectors]

,c10,c11,c12,c13
r70,0.007471,0.01315,0.00623,0.006137


In [51]:
share_consumption_based[list_sectors]

,c10,c11,c12,c13
share_cons_base,0.011717,0.007607,0.005379,0.002665


In [52]:
share_consumption_based[list_sectors].to_numpy()/df_share_sectors[list_sectors].to_numpy()

array([[1.56831421, 0.57848275, 0.86337257, 0.43422512]])

In [53]:
df_FDs["c57"].sum(axis=0)

41998603.67779265

In [54]:
df_Y.sum(axis=0)

Y    7.544723e+07
dtype: float64

In [55]:
df_FDs["c57"].sum(axis=0)/df_Y.sum(axis=0)

Y    0.556662
dtype: float64

# OUTPUTS: Export data to excel